In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

* meta
* desired_apriori
* PolynomialFeatures로 feature engineering
* target 의 mean이 0.03이라는 것으로 불균형
* VarianceThreshold 으로 low variance를 가진 feature을 없애는 기능
* assert

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

In [ ]:
train=pd.read_csv("/kaggle/input/porto-seguro-safe-driver-prediction/train.csv")
test=pd.read_csv("/kaggle/input/porto-seguro-safe-driver-prediction/test.csv")

# 1 Visual inspection of your data

In [ ]:
train.head(3)

In [ ]:
train.shape

In [ ]:
train=train.drop_duplicates()
train.shape

No duplicates

In [ ]:
test.shape

In [ ]:
train.info()

# 2 Defining the metadata

In [ ]:
data=[]
for f in train.columns:
    if f=='target':
        role='target'
    elif f=='id':
        role='id'
    else:
        role='input'
        
    if 'bin' in f or f=='target':
        level='binary'
    elif 'cat' in f or f=='id':
        level='nominal'
    elif train[f].dtype==float:
        level='interval'
    elif train[f].dtype==int:
        level='ordinal'
        
    keep=True
    if f=='id':
        keep=False
        
    dtype=train[f].dtype
    
    f_dict={
        'varname':f,
        'role':role,
        'level':level,
        'keep':keep,
        'dtype':dtype
    }
    data.append(f_dict)
    
meta=pd.DataFrame(data, columns=['varname', 'role','level','keep','dtype'])
meta.set_index('varname', inplace=True)

In [ ]:
meta.head(5)

meta는 행 번호를 거르기 위한 dataframe

In [ ]:
meta[(meta.level =='nominal') & (meta.keep)].index

In [ ]:
pd.DataFrame({'count': meta.groupby(['role','level'])['role'].size()}).reset_index()

# 3 Descriptive statistics

데이터 타입 별로 통계

### Interval variables

In [ ]:
v=meta[(meta.level == 'interval') & (meta.keep)].index #interval인 열만 가져오기, id는 아닌 것
train[v]

In [ ]:
train[v].describe()

### Reg variables
* range(최대~ 최소) -> standardscaler
* ps_reg_03 이 -1을 가짐

### Car variables

* ps_car_12,15가 -1

### Calc variables
* missing value 존재하지 않음
* calc인 열의 분포가 비슷하다

### Ordinal variables

In [ ]:
v=meta[(meta.level=='ordinal') & (meta.keep)].index
train[v].describe()

### Binary variables

In [ ]:
v=meta[(meta.level =='binary') & (meta.keep)].index
train[v].describe()

target 의 mean이 0.03이라는 것은 많이 imbalaned 됬다는 뜻이다.

# 4 Handling imbalanced classes

target=1 인게 0 보다 확연히 적다. 모델의 정확도를 높힐 수는 있으나 추가적인 가치는 없다(?)

* target=1을 oversampling
* target=0을 undersampling
이 두 가지로 문제를 해결할 수 있다

In [ ]:
desired_apriori=0.10
idx_0=train[train.target==0].index
idx_1=train[train.target==1].index

nb_0= len(train.loc[idx_0])
nb_1=len(train.loc[idx_1])
print(nb_0, nb_1)

In [ ]:
undersampling_rate=( (1-desired_apriori)*nb_1 )/( nb_0 * desired_apriori )
undersampled_nb_0=int(undersampling_rate*nb_0)
print('rate to undersample records with target=0: {}'.format(undersampling_rate))
print('number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

undersampled_idx=shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0) #섞어서 n_sample 만큼 뽑음
idx_list=list(undersampled_idx)+list(idx_1)
train=train.loc[idx_list].reset_index(drop=True) #그만큼 아예 삭제해버림

In [ ]:
print('target=0',len(train[train.target==0]))
print('target=1',len(train[train.target==1]))

# 5 Data Quality checks

### checking missing values

In [ ]:
vars_with_missing=[]

for f in train.columns:
    missings=train[train[f]==-1][f].count()
    #     mss=train[train[f]==-1].count() ==>x
    #     mss=train[f].loc[train[f]==-1].count() ==>o
    
    if missings>0:
        vars_with_missing.append(f)
        missings_perc=missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f,missings,missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

Variable ps_car_03_cat has 148367 records (68.39%) with missing values<br>
Variable ps_car_05_cat has 96026 records (44.26%) with missing values<br>
너무 많아서 아예 variable을 지운다

In [ ]:
vars_to_drop=['ps_car_03_cat','ps_car_05_cat']
train.drop(vars_to_drop,axis=1,inplace=True)
meta.loc[(vars_to_drop), 'keep']=False # 이제 셀 필요 없으니 meta에서도 false

* ps_reg_03: continuous : mean
* ps_car_11 : ordinal : mode
* ps_car_12 : continouous : mean
* ps_car_14 : continuous : mean

In [ ]:
mean_imp=SimpleImputer(missing_values=-1, strategy='mean')
mode_imp=SimpleImputer(missing_values=-1, strategy='most_frequent')

train['ps_reg_03']=mean_imp.fit_transform(train['ps_reg_03'].to_numpy().reshape(-1,1))
train['ps_car_12']=mean_imp.fit_transform(train['ps_car_12'].to_numpy().reshape(-1,1))
train['ps_car_14']=mean_imp.fit_transform(train['ps_car_14'].to_numpy().reshape(-1,1))

train['ps_car_11']=mode_imp.fit_transform(train['ps_car_11'].to_numpy().reshape(-1,1))

In [ ]:
# for f in train.columns:
#     missings=train[train[f]==-1][f].count()
#     print(f,missings)

### checking the cardinality of the categorical variables

cardinality: 한 feature안에서 다른 category 갯수
<br> 나중에 dummy feature을 만들 것이기 때문에 몇 가지 다른 값(카테고리)를 가지고 있는지 확인한다.

In [ ]:
v=meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values=train[f].value_counts().shape[0]
    print('variable {} has {} distinct values'.format(f, dist_values))

ps_car_11_cat 만 100개 넘는 다른 값들을 가지고 있다

In [ ]:
def add_noise(series, noise_level):
    return series * (1+noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, tst_series=None,
                 target=None, min_samples_leaf=1, smoothing=1, noise_level=0):
    assert len(trn_series) == len(target)
    assert trn_series.name==tst_series.name
    temp= pd.concat([trn_series, target], axis=1)
    
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    smotting = 1/(1+np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    prior=target.mean()
    averages[target.name] = prior * (1-smoothing) + averages["mean"]*smoothing
    averages.drop(["mean", "count"], axis=1,inplace=True)
    
    ft_trn_series = pd.merge(trn_series.to_frame(trn_series.name),
                            averages.reset_index().rename(columns={'index':target.name, target.name:'average'}),
                            on=trn_series.name,
                            how='left')['average'].rename(trn_series.name +'_mean').fillna(prior)
    ft_trn_series.index=trn_series.index
    ft_tst_series= pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
train_encoded, test_encoded = target_encode(train["ps_car_11_cat"],
                                           test["ps_car_11_cat"],
                                           target=train.target,
                                           min_samples_leaf=100,
                                           smoothing=10,
                                           noise_level=0.01)
train['ps_car_11_cat_te']=train_encoded
train.drop('ps_car_11_cat', axis=1, inplace=True)
meta.loc['ps_car_11_cat','keep']=False

test['ps_car_11_cat_te']=test_encoded
test.drop('ps_car_11_cat',axis=1,inplace=True)

# 6 Exploratory Data Visualization

## categorical variables

target=1인 고객들의 categorical variable을 알아보자

In [ ]:
v=meta[(meta.level=='nominal') & meta.keep].index

for f in v:
    plt.figure()
    fig, ax= plt.subplots(figsize=(20,10))
    cat_perc=train[[f,'target']].groupby([f], as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    print(cat_perc)
    
    sns.barplot(ax=ax, x=f, y='target', data=cat_perc, order=cat_perc[f])
    plt.ylabel('% target', fontsize=18)
    plt.xlabel(f, fontsize=18)
    plt.tick_params(axis='both',which='major', labelsize=18)
    plt.show()
    

그래프를 보니 missing values가 많이 차지하고 있어서 keep하는 것이 좋을 것 같다. 다른 예시로 대체하는 것보다 두는게 나을 것이다.

## Interval variables

In [ ]:
def corr_heatmap(v):
    correlations = train[v].corr()
    
    cmap=sns.diverging_palette(220, 10, as_cmap=True)
    
    fig,ax= plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f', square=True, linewidths=.5, annot=True, cbar_kws={"shrink":.75})
    plt.show()
    
v=meta[(meta.level=='interval') & (meta.keep)].index
print(v)
corr_heatmap(v)

* ps_reg_02 and ps_reg_03 (0.7)
* ps_car_12 and ps_car13 (0.67)
* ps_car_12 and ps_car14 (0.58)
* ps_car_13 and ps_car15 (0.67)

가 서로 관계가 깊으니까 pairplot으로 두 특징 사이의 관계를 알아보자

In [ ]:
s=train.sample(frac=0.1)
# 그래프 그리는데 시간을 높히기 위해서 샘플 사용

### ps_reg_02 and ps_reg_03

In [ ]:
sns.lmplot(x='ps_reg_02', y='ps_reg_03', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

### ps_car_12 and ps_car13

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

### ps_car_12 and ps_car14

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_14', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

### ps_car_13 and ps_car15

In [ ]:
sns.lmplot(x='ps_car_15', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

## checking the correlations between ordinal variables

In [ ]:
v=meta[(meta.level=='ordinal') & meta.keep].index
corr_heatmap(v)

# 7 Feature Engineering

### creating dummy variables

category 1과 2 가 나타내는게 2배 차이가 난다는 뜻이 아니므로 더미를 만들고 원래 feature을 지우자

In [ ]:
v=meta[(meta.level == 'nominal') & (meta.keep)].index
print('Before dummification we have {} variables in train'.format(train.shape[1]))

train=pd.get_dummies(train, columns=v, drop_first=True)
print('After dummification we have {} variables in train'.format(train.shape[1]))

### Creating interaction variables

In [ ]:
v=meta[(meta.level=='interval') & (meta.keep)].index
poly=PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
interactions=pd.DataFrame(data=poly.fit_transform(train[v]), columns=poly.get_feature_names(v))
interactions.drop(v, axis=1, inplace=True)

print('Before creating interactions we have {} variables in train'.format(train.shape[1]))
train=pd.concat([train, interactions], axis=1)
print('After creating interactions we have {} variables in train'.format(train.shape[1]))

In [ ]:
interactions

# 8 Feature Selection

### Removing features with low or zero variance

VarianceThreshold 으로 low variance를 가진 feature을 없애는 기능이 sklearn에 있다<br>
1% 아래의 variance를 삭제하면 31개를 삭제할 수 있다

In [ ]:
selector=VarianceThreshold(threshold=.01)
selector.fit(train.drop(['id', 'target'], axis=1))

f=np.vectorize(lambda x : not x) #boolean으로 나타남

v=train.drop(['id', 'target'], axis=1).columns[f(selector.get_support())]
#get_support 메서드는 선택된 특성을 불리언 값으로 표시해주어 어떤 특성이 선택되었는지 확인 할 수 있다.

print('{} variables have too low variance.'.format(len(v)))
print('These variables are {}.'.format(list(v)))

### Selecting features with a Random Forest and SelectfromModel

다른 classifier 에서 좋은 feature로 선택된 것을 선택하는 모델인데 random forest를 사용해보겠다.<br>
best feature 중 top 50%만 한다

In [ ]:
X_train=train.drop(['id','target'], axis=1)
y_train=train['target']
feat_labels=X_train.columns

rf=RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
rf.fit(X_train,y_train)

In [ ]:
importances=rf.feature_importances_
print(importances)
indices=np.argsort(rf.feature_importances_)[::-1]
print('indices', indices)

In [ ]:
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f+1, 30, feat_labels[indices[f]], importances[indices[f]] ))

In [ ]:
sfm=SelectFromModel(rf, threshold='median', prefit=True)
print("number of features before selection: {}".format(X_train.shape[1]))

n_features=sfm.transform(X_train).shape[1]
print('number of features after selection: {}'.format(n_features))
selected_vars=list(feat_labels[sfm.get_support()])

In [ ]:
train=train[selected_vars+['target']]

# 9 Feature Scaling

In [ ]:
scaler=StandardScaler()
scaler.fit_transform(train.drop(['target'], axis=1))